Normalização do modelo BCS não linear

In [49]:
import matplotlib.pyplot as plt
import numpy as np
exec(compile(open('param.py', "rb").read(), 'param.py', 'exec')) #% Roda arquivo com parâmetros do modelo BCS
%matplotlib tk

Dados carregados


### Variáveis de entrada
f $\in$ (35,65) Hz <br>
zc $\in$ (0,100)% <br>
pm $\in$ ($2\cdot 10^6$) <br>

### Variáveis de estado
pbh $\in$ ($10^5, 8.5 \cdot 10^6$) <br>
pwh $\in$ ($2 \cdot 10^6, 5.2\cdot10^6$) <br>
q $\in (10^{-2},3\cdot 10^{⁻2})$ <br>



Função para retornar os valores de xc e x0 dado que<br>
$x_{min}<x<x_{max}$ <br>

Nova variável<br>

$xb=\frac{x-x0}{xc}$

Assim <br>

$x=xb\cdot xc+x0$

Fazendo com que a nova variável esteja no intervalo $0<xb<1$ <br>


In [50]:
def Fnorm(xlim):
    # Encontrar o fator de normalização
    # tal que xb=(x-x0)/xc
    # xmin<x<xmax
    # fazendo com que 0<xb<1
    x=(xlim[0],xlim[1]-xlim[0])
    return x
def normalizar(x,xnorm):
    xs=np.zeros((nx,1))
    for i in range(0,nx):
        xs[i]=(x[i]-xnorm[i,0])/xnorm[i,1]
    return xs
def desnormalizar(x,xnorm):
    xs=np.zeros((nx,1))
    for i in range(0,nx):
        xs[i]=x[i]*xnorm[i,1]+xnorm[i,0]
    return xs
def AplicaEscala(var,i):
    aux=(var-xnorm[i,0])/xnorm[i,1]
    return aux
def normalizar_u(u,unorm):
    return ((u - unorm[:, 0]) / unorm[:, 1])

Definindo os fatores de ajuste de escala a partir dos limites operacionais

In [51]:
# #Teste

# t=np.arange(0,10,0.1)
# v=5*np.exp(-1*t)+3

# v=(v-3)/5
# plt.plot(t,v)

In [52]:
# Valores máximos e mínimos para normalização
#Entradas
f_lim=(30,75); zclim=(0,100);pmlim=(1e6,2e6);
pbhlim=(100000,8.5e6); pwhlim=(2e6,5.2e6); qlim=(12/3600,55/3600)
pbhlim=(3e6,12.6e6);
plim=(1e5,12.6e6) # pressão característica
pbhlim=plim
pwhlim=plim




In [53]:
pm=2e6; #Simplificando pm fixo
unorm=np.array([Fnorm(f_lim),Fnorm(zclim)])
xnorm= np.array([Fnorm(pbhlim),Fnorm(pwhlim),Fnorm(qlim), unorm[0,:],unorm[1,:]])
#pbh  - pressão na bomba
#pwh - Pressão de fundo do poço,
#q - vazão
#PI índice de produtividade do poço
#PinC  pressão na choke
# Pressão de manifold

Definindo as variáveis simbólicas

In [54]:
# Criando simbolica
from casadi import *
nx = 3; nu = 2;
x = MX.sym("x",nx); # Estados
u = MX.sym("u",nu); # Exogena
dudt_max = MX.sym("dudt_max",2); # Exogena


In [55]:
pbh = x[0]
pwh = x[1]
q = x[2]


In [56]:
# Entradas

fqref = u[0]; zcref = u[1];
pm=2e6;
[pr,PI]

[12600000.0, 2.32e-09]

#Simplificando modelo do matlab de bruno. Entradas não serão estados


In [57]:
zc=zcref
fq=fqref

In [58]:
tc=1/(b1/V1*PI)
F1c=941799.5331
F2c=2260318.8795
Ppc=14090869.6942
pbc=pr
qc1=pbc
Hc=1511.97
pwc=tc*qc1*b2/V2
qcc=pwc*V2/tc/b2
print(qcc)
#qcc= 0.033987702
pbh=pbc*pbh
pwh=pwc*pwh
q=q*qc1
pbh
#fq=fc*fq # duvida verificar?

12600000.0


MX((12600000*x[0]))

MX((1.66759e+15*x[1]))

In [59]:
# Escalonando parâmetros que se relacionam diretamente com variáveis escalonadas previamente na mesma escala
# pm=(pm-xnorm[1,0])/xnorm[1,1]
# pr=(pr-xnorm[0,0])/xnorm[0,1]
# f0=(f0-unorm[0,0])/unorm[0,1]
# [pr,pm,f0]

Normalizando variáveis dependentes

In [60]:
# Calculo do HEAD e delta de press�o
q0 = q/Cq*(f0/fq);
H0 = -1.2454e6*q0**2 + 7.4959e3*q0 + 9.5970e2;
H = CH*H0*(fq/f0)**2; # Head
Pp = rho*g*H;
# Calculo da press�o de intake
F1 = 0.158*((rho*L1*q**2)/(D1*A1**2))*(mu/(rho*D1*q))**(1/4);
F2 = 0.158*((rho*L2*q**2)/(D2*A2**2))*(mu/(rho*D2*q))**(1/4);
funcF1=Function('funcF1',[x],[F1])

#[funcF1(np.array([0,0,0.01,0,0])),funcF1(np.array([0,0,1,0,0]))]

In [61]:
P0 = -2.3599e9*q0**3 -1.8082e7*q0**2 +4.3346e6*q0 + 9.4355e4;
pin = pbh - rho*g*h1 - F1
P = Cp*P0*(fq/f0)**3; # Potencia
I = Inp*P/Pnp;      # Corrente

In [62]:
# Vazao do reservatorio e vazao da choke
qr  = PI*(pr - pbh);

In [63]:
qc  = Cc*(zc/100)*casadi.sign((pwh - pm))*casadi.sqrt(casadi.fabs(pwh - pm));
#pr*b1*PI/V1

In [64]:
qc=qc*qcc
Pp=Pp*Ppc
F1=F1c*F1
F2=F2c*F2
H=Hc*H
#qr=qrc*qr

dpbhdt = (tc/pbc)*b1/V1*(qr - q)
dpwhdt = (tc/pwc)*b2/V2*(q - qc)
dqdt = (tc/qc1)*1/M*(pbh - pwh - rho*g*hw - F1 - F2 + Pp);

# #Novo vetor de estados
# xn = MX.sym("xn",nx); # Estados
# un = MX.sym("un",nu); # Exogena




$$$$\dot{\overline{pbh}}=\frac{t_cb_1}{V_1pb_c}PIP_r-\frac{t_cb_1\cdot  PI}{ V_1}\overline{pbh}-\overline{q}\frac{q_c}{pb_c}$$
$$\dot{\overline{pwh}}=\frac{t_cb_2}{pw_cV_2}q_c\overline{q}-\frac{t_cb_2}{pw_cV_2}q_{cc}\overline{q}_{ch}(\overline{pwh}pw_c))$$

$$\dot{\overline{q}}=\overline{pbh}\frac{t_cpb_c}{q_c}  -\overline{pwh}\frac{t_cpw_c}{q_c}  -\rho gh_w\frac{t_c}{q_c} -\overline{F_1}(\overline{q}q_c)\frac{t_c{F_1}_c}{q_c} - \overline{F_2}(\overline{q}q_c)\frac{t_c{F_2}_c}{q_c}+\rho g H_c\frac{t_c}{q_c}\overline{H}(f_c\overline{f},q_c\overline{q})$$

$F1_c=941799.53$, $F2_c=2260318.8795$, $Hc=1511.97$ e $g_{cc}=0,033987702$

In [65]:
print('Coeficientes')
print('eq1')
print([tc*b1*PI*pr/V1/pbc,tc*b1*PI/V1, qc1/pbc])
print('eq2')
print([tc*b2*qc1/V2/pwc,tc*b2*qcc/V2/pwc])
print('eq3')
print([tc*pbc/qc1,tc*pwc/qc1,rho*g*hw*tc/qc1,F1c*tc/qc1,F2c*tc/qc1,rho*g*Hc*tc/qc1])



Coeficientes
eq1
[1.0, 1.0, 1.0]
eq2
[0.9999999999999998, 0.9999999999999998]
eq3
[1.1649425287356323, 209233903.06151247, 0.861641420361248, 0.08707478806758348, 0.20897949136774768, 1.3027759783435964]


In [66]:
c.

SyntaxError: invalid syntax (<ipython-input-66-cf463a5954f9>, line 1)

Aplicar escala nas derivadas

In [17]:
# dpbhdt=dpbhdt/xnorm[0,1]
# dpwhdt=dpwhdt/xnorm[1,1]
# dqdt=dqdt/xnorm[2,1]

In [62]:
# Função casadi
dxdt = casadi.vertcat(dpbhdt,dpwhdt,dqdt,0,0)
Eq_Estado = casadi.Function('Eq_Estado',[x,u],[dxdt],
                     ['x','u'],['dxdt'])

RuntimeError: Error in Function::Function for 'Eq_Estado' [MXFunction] at .../casadi/core/function.cpp:235:
.../casadi/core/function_internal.cpp:144: Error calling MXFunction::init for 'Eq_Estado':
.../casadi/core/x_function.hpp:286: For Eq_Estado: Xfunction input arguments must be purely symbolic.
Argument 0(x) is not symbolic.

In [19]:
y=vertcat(pin,H);
ny = y.size1()
# Equações algébricas
sea_nl = casadi.Function('sea_nl',[x,u],[y,pin,H],\
                 ['x','u'],['y','pin','H']); # Sistema de Eq. Algebricas variaveis de sa�da

In [20]:
BCS={
     'x': x,
     'u': u,
     'y': y,
     'nx': nx,
     'nu': nu,
     'ny': ny,
     'NaoLinear': {'sedo_nl': Eq_Estado(x,u),
                   'sea_nl': sea_nl
                   }
}

In [22]:
#% Otimizador
opt={
     'ipopt':{
         'print_level':0,
         'acceptable_tol':1e-8,
         'acceptable_obj_change_tol':1e-6,
         'max_iter':50
         },
     'print_time':0,
     }

opt['ipopt']['print_level']=0;# %0,3
opt['print_time']=0;
opt['ipopt']['acceptable_tol']=1e-8;
opt['ipopt']['acceptable_obj_change_tol']=1e-6;
opt['ipopt']['max_iter']=50;

In [23]:
MMQ = {'x':BCS['x'], 'f':J, 'p':BCS['u']}
#nlp={'x':vertcat(BCS['x'],BCS['u']), 'f':J} #variáveis de decisão, função f, g (N/A)
#nlp={'x':BCS['x'], 'f':J}
solver = nlpsol('solver', 'ipopt', MMQ, opt)
# Restrições das variaveis de decis�o
# minimo
args={
      'lbx': np.zeros((nx,1)),
# m�ximo
      'ubx':np.full((nx, 1), np.inf)
      }

# Solu��o do otimizador
sol=solver(x0=BCS['x'], lbx=args['lbx'], ubx=args['ubx'], p=BCS['u']);
yss=sea_nl(sol['x'],BCS['u'])
Estacionario = Function('Estacionario',[BCS['x'],BCS['u']],\
    [sol['x'],yss[0]],\
    ['x0','uss'],['xss','yss']);

BCS['Estacionario'] = Estacionario;
f_ss,zc_ss= (np.array([50, 50]))
uss = np.array([f_ss,zc_ss]); # Entradas do estacionario
uss_n=normalizar_u(uss,unorm)
uss_n

In [29]:
#% Calculo do estacionario

#x0 = [0.2,0.5,0.5,0.5,0.5]
x0 = np.array([8311024.82175957,2990109.06207437,0.00995042241351780,50,50]);

In [30]:
x0_n=normalizar(x0,xnorm)
x0_n

array([[0.65688199],
       [0.23120872],
       [0.55398885],
       [0.44444444],
       [0.5       ]])

In [31]:
# args['lbx'][3] = uss_n[0]
# args['ubx'][3] = uss_n[0]  # bounds freq. solver args['lbx'][4] = uss[1]
# args['ubx'][4] = uss_n[1]   # bounds zc solver
sol=solver(x0=x0_n, p=uss_n);
#sol=solver(x0=x0_n,lbx=args['lbx'], ubx=args['ubx'], p=uss_n)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



In [32]:
xss_n=sol['x']

In [33]:
np.array(sol['x'])

array([[0.65688245],
       [0.23120831],
       [0.00335081],
       [0.44444444],
       [0.5       ]])

In [34]:
xss = desnormalizar(sol['x'],xnorm)

In [35]:
xss

array([[8.31103064e+06],
       [2.99010385e+06],
       [3.37335695e-03],
       [5.00000000e+01],
       [5.00000000e+01]])

In [36]:
x0.reshape(5,1)
sol['x']

DM([0.656882, 0.231208, 0.00335081, 0.444444, 0.5])

In [37]:
pin

MX(@1=((x[2]-0.00333333)/0.0119444), ((((x[0]-100000)/12500000)-1863900)-((0.158*((475000*sq(@1))/6.6775e-06))*pow((0.025/(96.52*@1)),0.25))))

In [38]:
yss=sea_nl(xss_n,uss_n)[0]
yss


DM([-1.87951e+06, 672.668])

In [39]:
# Definir variaveis manipuladas e controladas e disturbio externo
mv = [0,1]    #% [f, Zc]
pv = [0,1]  #% [pin, H]  #% [P, I]
#pv = [2,3];  #% [pin, H]
de = 2;      #% [pm]
tg = 2;      #% MV target
#% Parametros
ts = 1;
BCS['u'].shape

(2, 1)

In [40]:
#%Modelo de predição
#% Criando o objeto para predição do modelo
# Iniciando variavel dicionário para a construção da EDO
# sedo = {'x': BCS['x'][0:3], # Estados
#         'p': BCS['u'], #Variáveis exogenas
#         'ode': BCS['NaoLinear']['sedo_nl'] # SEDO (Gerado no bcs_settings)
#         };

sedo = {'x': BCS['x'], # Estados
        'p': BCS['u'], #Variáveis exogenas
        'ode': Eq_Estado(x,u) # SEDO (Gerado no bcs_settings)
        };

#% Criando o objeto p,ra integração da Eq_estado
opt = {'tf':ts,
       't0':0

       };   #% opções do integrador


In [41]:
int_odes = integrator('int_odes','cvodes',sedo,opt);

In [42]:
# objeto integrador
res = int_odes(x0=BCS['x'],p=BCS['u']);             #   % solução um passo a frente
npv = len(pv); nmv = len(mv);

In [43]:
# Criando o objeto para solução da equação de medição
Eq_medicao = Function('Eq_medicao',[BCS['x'],BCS['u']],[BCS['y'][pv]],['x','u'],['y']);
# Criacao do objeto para simulacao do BCS Eq de estado + Eq de Medicao
Modelo_Predicao = Function('Modelo_Predicao',[BCS['x'],BCS['u']],[res['xf'],Eq_medicao(res['xf'],BCS['u'])],['xk_1','uk_1'],['xk','yk']);
Modelo_Predicao2 = Function('Modelo_Predicao2',[BCS['x'],BCS['u']],[res['xf']],['xk_1','uk_1'],['xk']);

In [44]:
# Inicializa��o das variaveis
xss=x0_n
xpk = xss;
uss

array([50, 50])

In [45]:
#xpk=normalizar(xss,xnorm)
xpk=(xss-xnorm[:,0])/xnorm[:,1]
#uk_1 = normaliza_u(uss[mv],unorm)
#uk_1=(uss[mv]-unorm[:,0])/unorm[:,1]
# MVS
# Aloca��o de variaveis
Xk = np.zeros((nx,1))

In [46]:
Yk = np.zeros((npv,1))
Uk = np.zeros((nmv,1))
Ymk = Yk
Ys = Yk
Ymin = Yk
Ymax = Yk

### Simulando cenários

In [47]:
tsim = 100;
nsim=int(round(tsim/ts)+1)
xss_n=x0_n
x0_n

array([[0.65688199],
       [0.23120872],
       [0.55398885],
       [0.44444444],
       [0.5       ]])

In [48]:
#Valores iniciais de simulação
tss=5 # Tempo inicial no estacionário
Uk= uss
# Povoar vetor de X com dados não normalizados

In [49]:
x0_n

array([[0.65688199],
       [0.23120872],
       [0.55398885],
       [0.44444444],
       [0.5       ]])

In [50]:
xpk = Modelo_Predicao2(x0_n,uss_n) #valores normalizados

ypk=sea_nl(xpk,uss_n)[0]
Yk=ypk

Xk=desnormalizar(xpk,xnorm)
# for i in range(0,tss):
#     Xk = hcat([Xk,desnormalizar(xpk,xnorm)])
#     Yk = hcat([Yk,np.array(ypk)]);
#     Uk = hcat([Uk,uss]);

uk_1 = np.array([[60], [70]]);
uk_1n=(uk_1.T-unorm[:,0])/unorm[:,1]

output ode, at (row 2, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-06-17 21:08:56 WARNING("int_odes:jacF failed: NaN detected for output o0, at nonzero index 6 (row 2, col 2).") [.../casadi/core/oracle_function.cpp:265]
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
CasADi - 2021-06-17 21:08:56 WARNING("int_odes:odeF failed: NaN detected for output ode, at (row 2, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-06-17 21:08:56 WARNING("int_odes:jacF failed: NaN detected for output o0, at nonzero index 6 (row 2, col 2).") [.../casadi/core/oracle_function.cpp:265]
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
CasADi - 2021-06-17 21:08:56 WARNING("int_odes:odeF failed: NaN detected for output ode, at (row 2, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-06-17 21:08:56 WARNING("int_odes:jacF failed: NaN detected for output o0, at 

RuntimeError: Error in Function::operator() for 'int_odes' [CvodesInterface] at .../casadi/core/function.cpp:1368:
.../casadi/interfaces/sundials/cvodes_interface.cpp:392: CVode returned "CV_TOO_MUCH_WORK". Consult CVODES documentation.

In [121]:
xi=(np.arange(0,int(nsim*ts)-1,ts));
#xi=np.arange(0,10,ts)
xi.shape

(100,)

In [122]:
print("Simulação iniciada")

#for k in range(1,10):
for k in range(1,nsim-1):
    #print('Tempo:',k*ts, 'k=',k)
    xpk = Modelo_Predicao2(xpk,uk_1n)
    #print(xpk[0:3])
    #print(desnormalizar(xpk,xnorm))
    Xk = hcat([Xk,desnormalizar(xpk,xnorm)])
    #print(Xk.shape)
    ypk=sea_nl(xpk,uk_1n)[0]
    #y2=sea_nl(desnormalizar(xpk,xnorm),uk_1)
    #y2
    Yk = hcat([Yk,np.array(ypk)]);
    #Yk = hcat([Yk,y2]);
    Uk = hcat([Uk,uk_1])

    #Ymk = hcat([Ymk,ymk]);
    #print("Xk,Yk,Uk,xi shape ="+str(Xk.shape)+" "+str(Yk.shape)+" "+str(Uk.shape)+" "+str(xi.shape))
print("Xk shape ="+str(Xk.shape))
# print("Uk shape ="+str(Uk.shape))
# print("Yk shape ="+str(Uk.shape))
# print("xi shape ="+str(xi.shape))
print("Ok.simulação concluida")
Xk[0,:].shape

Simulação iniciada
Xk shape =(5, 100)
Ok.simulação concluida


(1, 100)

In [123]:
np.arange(0,tss*ts,ts)

array([0, 1, 2, 3, 4])

In [124]:
xi.shape

(100,)

In [125]:
#xi=vcat([np.arange(0,tss*ts,ts),xi])
xi.shape

(100,)

## Produção dos gráficos

In [126]:
fig1=plt.figure()
label = [r'$p_{in}(bar)$','H(m)','P','I','qc','qr' ];
for iy in range(0,npv):
    ax = fig1.add_subplot(npv,1,iy+1)
    #print(iy)
    if iy == 0: # Pin
        ax.plot(xi,(Yk[iy,:].T), label='Medição')
        #ax.plot(xi,Ymk[iy,:].T/1e5, label='EKF')
        ax.set_ylabel(label[iy])
        ax.set(xlim=(xi[0], nsim*ts))
       # ax.set(ylim=(40,62))
        plt.grid(True)
    else: # H
        ax.plot(xi,Yk[iy,:].T, label='Medição')
        #ax.plot(xi,Ymk[iy,:].T,label='EKF')
        ax.set_ylabel(label[iy])
        ax.set(xlim=(xi[0], nsim*ts))
       # ax.set(ylim=(580, 850))
        plt.grid(True)
#ax.plot(xi,Yk[2,:].T, label='EKF')
ax.legend();
ax.set_xlabel('Time (nT)')
fig1.show()


In [127]:
# Yk[1,:].T.shape
#

In [128]:
# #% Restrição
umin  = np.array([35, 0]); np.transpose(umin);  # lower bounds of inputs
umax  = np.array([65, 100]); np.transpose(umax); # upper bounds of inputs
dumax = np.array([0.5, dzc_max]); np.transpose(dumax);
fig2=plt.figure()
label = ['f(Hz)',r'$z_c$(%)'];
for iu in range(0,nmv):
    ax2=fig2.add_subplot(nmv,1,iu+1)
    ax2.plot(xi,Uk[iu,:].T, label='Medição')
    ax2.plot([1,nsim],[umin[iu], umin[iu]],'--r')
    ax2.plot([1,nsim],[umax[iu], umax[iu]],'--r', label='Restrição')
    ax2.set_ylabel(label[iu])
    ax2.set(xlim=(xi[0], nsim*ts))
    if iu==0:
        ax2.set(ylim=(30, 70))

    plt.grid(True)


In [129]:
fig3=plt.figure()
label = ['Pbh','Pwh','q'];
for iu in range(0,3):
    ax3=fig3.add_subplot(3,1,iu+1)
    ax3.plot(xi,Xk[iu,:].T, label='Medição')
    #ax3.plot([1,nsim],[umin[iu], umin[iu]],'--r')
    #ax3.plot([1,nsim],[umax[iu], umax[iu]],'--r', label='Restrição')
    ax3.set_ylabel(label[iu])
    #ax3.set(xlim=(xi[0], nsim*ts))
    # if iu==0:
    #     #ax2.set(ylim=(30, 70))
    #     print(iu)
    plt.grid(True)

In [130]:
# fig1=plt.figure()
# label = [r'$p_{in}(bar)$','H(m)','P','I','qc','qr' ];
# Yk=np.array(Yk).reshape(100,2)
# plt.plot(xi,np.array(Yk[:,0]))
# plt.plot(xi,np.array(Yk[:,1]))

In [131]:
# ax = fig1.add_subplot(2,1,1)
# #ax.set_xlabel('Time (nT)')
# ax.plot(xi,np.array(Yk[:,0]))
# ax = fig1.add_subplot(2,1,2)
# ax.plot(xi,Yk[:,1])
# plt.grid(True)
# fig1.show()

In [132]:
# [Yk[:,0].shape, xi.shape]